# High Performance Python

### by Ankush Chander
Research Engineer

## Table of contents:
1. The Fundamental Computer System
2. Profiling techniques
3. Python lists
4. Dictionary and namespaces












In [1]:
! pip install memory_profiler
%load_ext memory_profiler
# for %timeit and %memit


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


## Fundamental Computer System

### Computing units
Provides the ability to transform bits it recieves  into other bits or to change the state of process.
Eg: CPU(commonly used), GPU(auxilary unit) 

### Memory units
Used to store bits.
Eg: Spinning disc, Solid State Drive(SSD), RAM, l1/l2/l3 cache, CPU registers  

### Communication layers
How fast we can move data from one place to another
Eg: Frontside bus(between RAM and l1/l2 cache), External bus(hardware devices to CPU and main memory) 


## Computing units
Provides the ability to transform bits it recieves  into other bits or to change the state of process.

#### Measurements:

How many cycles per second?(Clock speed)  
A CPU cycle is the basic unit of time used by a CPU to execute an instruction. It consists of a fetch, decode, execute, and write-back phase.

How many instructions per cycle(IPC)?  
Having a high IPC can drastically affect computing by changing the level of vectorization that is possible. 


#### Commands:
```
lscpu
eg: output
CPU MHz:                         1700.000 # 1 MHz means 1 million cycles per second
CPU max MHz:                     4700.0000
CPU min MHz:                     400.0000
BogoMIPS:                        3379.20  # Million instructions per second
```

### Memory units
Used to store bits.
#### Measurements
1. How much data it can hold? 
2. How fast it can read/write to it.
3. How fast it take to locate data 

#### Tiered structure:
1. Spinning hard drive
2. Solid State hard drive
3. RAM
4. l1/l2/l3 cache
5. CPU registers

## Relative time latency 
![Memory access](img/relative-time-latencies-computer-programming.jpg)  
Image credits: [Relative time latencies and computer programming](https://alvinalexander.com/photos/relative-time-latencies-and-computer-programming/)

## Communication layers
How fast we can move data from one place to another

### Measurements:
How much data can a bus move in transfer?(Bus width)
How many transfers bus can make in 1 second(bus frequency)

### Types of bus:
1. Frontside bus: connects RAM to l1/l2 cache
2. External bus: connect external devices to system memory/CPU
3. PCI bus: For peripheral devices


## Language choices
### Python is a garbage-collected language 
Memory is automatically allocated and freed when needed.
Leads to mempory fragmentation.
### Dynamic types and language not being compiled
If code is static than compiler can make lot of optimization to lay out objects in memory efficiently so that CPU run certain instructions more efficiently. Python being dynalic language reduces the scope of such optimzation. 
### Global interpretor lock(GIL)
Python handles race condtions and deadlocks using GIL. GIL makes sure that a Python process can run only one instruction at a time regardless of number of cores it is currently using.

## Why use python?
1. It"s "batteried included" language with key features either builtin or accessible via external libraries.
2. Managed environments and shells available.
3. It"s faster to prototype ideas in python.

## Profiling techniques
1. Simple approaches  
2. linux time command  
3. line_profiler  
4. perf module

### Simple approaches to timing - print and decorator

In [22]:
# time module
import time

s_time = time.time()
time.sleep(4)
print(f"took {time.time() - s_time} secs")

took 4.000414609909058 secs


In [25]:
# log function"s execution time using decorator

import time
from functools import wraps

def timer_func(func):
    # This function shows the execution time of 
    # the function object passed
    @wraps(func)
    def wrap_func(*args, **kwargs):
        t1 = time.time()
        result = func(*args, **kwargs)
        t2 = time.time()
        print(f'Function {func.__name__!r} executed in {(t2-t1):.4f}s')
        return result
    return wrap_func
  

@timer_func
def lengthy_function():
    time.sleep(4)
    
    
lengthy_function()

### Simple Timing Using the Unix `time` Command

In [26]:
! time python3 -u high_performance_python_examples/insertion_sort.py  10000


real	0m3.258s
user	0m3.249s
sys	0m0.008s


### line_profiler
```
# installation: 
pip install line_profiler

# decorate functions to be profile using @profile

# how to run
kernprof -l -v  python_script.py
```

line_profiler provides following info corresponding to each line

Line: Line number

Hits: number of times line got executed

Per Hit: approx time taken by that line

% Time: percentage time spent executing the line

Line Contents: Line content

%time is most useful parameter in case looking for CPU bottlenecks

In [5]:
! kernprof -l -v  high_performance_python_examples/insertion_sort.py

Wrote profile results to insertion_sort.py.lprof
Timer unit: 1e-06 s

Total time: 0.004956 s
File: high_performance_python_examples/insertion_sort.py
Function: insertion_sort at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           @profile
    14                                           def insertion_sort(arr):
    15                                               # Traverse through 1 to len(arr)
    16        99         31.0      0.3      0.6      for i in range(1, len(arr)):
    17        98         33.0      0.3      0.7          key = arr[i]
    18                                                   # Move elements of arr[0..i-1], that are
    19                                                   # greater than key, to one position ahead
    20                                                   # of their current position
    21        98         32.0      0.3      0.6          j = i - 1
    22      4949       1784.0  

### perf module
Probing how well memory is being moved to the CPU can be quite hard; however, in
Linux the perf tool can be used to get amazing amounts of insight into how the CPU
is dealing with the program being run.

```
# installation:
sudo apt update
sudo apt install linux-tools-common

# how to run?
sudo perf stat -e cycles,stalled-cycles-frontend,stalled-cycles-backend,instructions,cache-references,cache-misses,branches,branch-misses,task-clock,faults,minor-faults,cs,migrations -r 3 python3 high_performance_python_examples/insertion_sort.py 

# cycles:  tells us how many clock cycles our task took.
#instructions:  tells us how many instructions our code is issuing to the CPU.
#cache-references: data attempted to be referenced via cache
#cache-misses: data references not found in cache and was fetched from RAM
#branches: a time in the code where the execution flow changes. eg: if..else 
#branch-misses: cpu tries to predict the flow and preload the instructions. when prediction is incorrect.
#faults: memory allocations
```


## Strategies for profiling code 
1. Disable TurboBoost in the BIOS.

2. Disable the operating system’s ability to override the SpeedStep (you will find this in your BIOS if you’re allowed to control it).

3. Only use mains power (never battery power).

4. Disable background tools like backups and Dropbox while running experiments.

5. Run the experiments many times to obtain a stable measurement.

6. Possibly drop to run level 1 (Unix) so that no other tasks are running.

7. Reboot and rerun the experiments to double-confirm the results.

## Data structures and tradeoffs and quirks
1. Dynamic lists and caching
2. Resizing and memory allocation
3. Difference between python lists and Numpy arrays
4. Performance analysis of dictionaries

### Lists
Python lists are dynamic in nature.

#### Pros: 
1. User dont need to know list size before hand.
2. User don"t have to handle resizing
#### Cons:
1. Resizing(in background) makes appending to array slower.  
2. Due to resizing lists usually take more space than the elements it holds.

In [27]:
%timeit [i*i for i in range(10000)]

267 µs ± 3.97 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [28]:
%%timeit l = []
for i in range(10000):
    l.append(i*i)
    
# takes more time because of list resizing

458 µs ± 7.55 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [29]:
%memit [i*i for i in range(1000000)]

peak memory: 161.44 MiB, increment: 0.00 MiB


In [30]:
%%memit l = []
for i in range(1000000):
    l.append(i*i)
    
# takes more time because of list resizing

peak memory: 159.16 MiB, increment: 27.62 MiB


## Key Factors
### Memory allocations: 
Memory allocations are not cheap. Whenver you assign a value to variable interpreter needs to talk to Operating system to allocate new space and then iterate over that space to access it. Try to allocate space fewer times and then reuse that space throughout the program.

### Memory fragmentation:
Python lists doesn"t natively support vectorization because lists doesnt store the actual data but pointers to it.

### Cache misses
Since data is fragmented across different memory locations, you must move each piece over instead of a block over, hence missing out on benifits of caching


## Lists vs numpy arrrays

In [10]:
from array import array
import numpy as np


def norm_square_list(vector):
    norm = 0
    for v in vector:
        norm += v*v
    return norm


def norm_square_list_comprehension(vector):
    return sum([v*v for v in vector])


def norm_square_numpy(vector):
    return np.sum(vector * vector)


def norm_square_numpy_dot(vector):
    return np.dot(vector, vector)



In [11]:
%%timeit
vector = list(range(1000000))
norm_square_list(vector)

47.8 ms ± 4.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [12]:
%%timeit
vector = list(range(1000000))
norm_square_list_comprehension(vector)

57.5 ms ± 576 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
vector_numpy = np.arange(1000000)
norm_square_numpy(vector_numpy)
# numpy has specialized code in C that takes benifits of any vectorization that CPU has enabled. 
#Also numpy arrays are represented sequentially in memory as low level data types

957 µs ± 6.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit
vector_numpy = np.arange(1000000)
norm_square_numpy_dot(vector_numpy)
# uses specialized code that doesn"t need to store intermediate output of (vector * vector)

975 µs ± 167 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Dictionaries and namespaces
Python’s namespace management, which heavily uses dictionaries to do its lookups.
Whenever a variable, function, or module is invoked in Python, there is a hierarchy that
determines where it looks for these objects. First, Python looks inside of the **locals()
array**, which has entries for all local variables. Python works hard to make local variable
lookups fast, and this is the only part of the chain that doesn’t require a dictionary
lookup. If it doesn’t exist there, then the **globals() dictionary** is searched. Finally, if the
object isn’t found there, the **__builtin__ object** is searched.

In [15]:
import math
from math import sin
import dis

def test1(x):
    """
    resolve `math` from globals()
    resolve `sin` from math module
    local lookup for x
    
    """
    return math.sin(x)


def test2(x):
    """
    resolve sin from globals()
    resolve x from locals()
    """
    return sin(x)


def test3(x, sin=math.sin):
    """
    resolve sin from locals()
    resolve x from locals()
    """
    return sin(x)

In [16]:
%timeit test1(123456) 
#2 dictionary lookup, 1 local lookup


55.2 ns ± 0.442 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [17]:
%timeit test2(123456)
#1 dictionary lookup, 1 local lookup

52.4 ns ± 0.087 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [18]:
%timeit test3(123456)
# 2 local lookups

52.7 ns ± 0.224 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [19]:
dis.dis(test1)

  5           0 RESUME                   0

 12           2 LOAD_GLOBAL              0 (math)
             14 LOAD_METHOD              1 (sin)
             36 LOAD_FAST                0 (x)
             38 PRECALL                  1
             42 CALL                     1
             52 RETURN_VALUE


In [20]:
dis.dis(test2)

 15           0 RESUME                   0

 20           2 LOAD_GLOBAL              1 (NULL + sin)
             14 LOAD_FAST                0 (x)
             16 PRECALL                  1
             20 CALL                     1
             30 RETURN_VALUE


In [21]:
dis.dis(test3)

 23           0 RESUME                   0

 28           2 PUSH_NULL
              4 LOAD_FAST                1 (sin)
              6 LOAD_FAST                0 (x)
              8 PRECALL                  1
             12 CALL                     1
             22 RETURN_VALUE


## Further exploration
1. How can we write modules for python using C/fortran/rust
2. Multiprocessing
3. How can we write code that runs across a cluster and use more machines.
4. How can we make programs use less RAM

## References
1. [High Performance Python(book)](https://www.goodreads.com/book/show/49828191-high-performance-python)
2. [What Is Clock Speed?](https://www.intel.com/content/www/us/en/gaming/resources/cpu-clock-speed.html)
3. [BogoMIPS](https://en.wikipedia.org/wiki/BogoMips)
4. [Global Interpretor Lock](https://realpython.com/python-gil/)
5. [dis — Disassembler for Python bytecode](https://docs.python.org/3/library/dis.html)
6. [Cython and regular expressions](https://romankleiner.blogspot.com/2015/06/cython-and-regular-expressions.html)
7. [How can node.js be faster than c and java? Benchmark comparing node.js, c, java and python](https://stackoverflow.com/questions/39360403/how-can-node-js-be-faster-than-c-and-java-benchmark-comparing-node-js-c-java)